In [76]:
import copy
import numpy as np
import cv2
import csv
import os
import itertools
import matplotlib.pyplot as plt
import PIL

from transformers import pipeline

import tensorflow as tf

import mediapipe as mp 
from mediapipe.tasks import python
from mediapipe.tasks.python import vision



In [77]:
mp_hands = mp.solutions.hands

hands = mp_hands.Hands(
    max_num_hands = 2,
    min_detection_confidence = 0.7,
    min_tracking_confidence = 0.5,
)

I0000 00:00:1725835513.281816  957293 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M1 Pro


W0000 00:00:1725835513.290706 1052447 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1725835513.299404 1052447 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [78]:
def calc_bounding_box(image, landmarks):
    image_width, image_height = image.shape[1], image.shape[0]

    landmark_array = np.empty((0,2), int)

    for _, landmark in enumerate(landmarks.landmark):
        landmark_x = min(int(landmark.x * image_width), image_width - 1)
        landmark_y = min(int(landmark.y * image_height), image_height - 1)
        landmark_point = [np.array((landmark_x, landmark_y))]

        landmark_array = np.append(landmark_array, landmark_point, axis = 0)

    x,y,w,h = cv2.boundingRect(landmark_array)
    return [x,y,x+w,y+h]

In [79]:
def draw_bounding_box(image, brect):
    cv2.rectangle(image,(brect[0], brect[1]), (brect[2], brect[3]), (0,0,0), 1)

    image = depth_map_calculator(image, brect)
    return image

In [48]:
estimator = pipeline(task = "depth-estimation", model = "Intel/dpt-large")

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of DPTForDepthEstimation were not initialized from the model checkpoint at Intel/dpt-large and are newly initialized: ['neck.fusion_stage.layers.0.residual_layer1.convolution1.bias', 'neck.fusion_stage.layers.0.residual_layer1.convolution1.weight', 'neck.fusion_stage.layers.0.residual_layer1.convolution2.bias', 'neck.fusion_stage.layers.0.residual_layer1.convolution2.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration. P

In [75]:
def depth_map_calculator(image, brect):

    roi = image[brect[1]:brect[3], brect[0]:brect[2]]
    roi = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)
    roi = PIL.Image.fromarray(roi)

    output = estimator(images = roi)
    depth_image = output['depth']
    depth_image = np.asarray(depth_image)
    depth_image = cv2.cvtColor(depth_image, cv2.COLOR_BGR2RGB)

    image[brect[1]:brect[3], brect[0]:brect[2]] = depth_image

    return image

In [80]:
vid = cv2.VideoCapture(0)
while (True):

    key = cv2.waitKey(10)
    if key == 27:
        break
    # Camera capture
    ret, image = vid.read()
    # Stopping the loop camera stops detecting
    if not ret:
        break
    # Flipping the image as the camera capture is mirrored.
    image = cv2.flip(image,1)
    debug_image = copy.deepcopy(image)
    # Hand detection
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = hands.process(image)

    if results.multi_hand_landmarks:
        for hand_landmarks, handedness in zip(results.multi_hand_landmarks, results.multi_handedness):

            # Assigning bounding box
            brect = calc_bounding_box(debug_image, hand_landmarks)
            # Drawing bounding box
            debug_image = draw_bounding_box(debug_image, brect)

    cv2.imshow('Hand Gesture Recognition', debug_image)

vid.release()
cv2.destroyWindow('Hand Gesture Recognition')
cv2.waitKey(1)

error: OpenCV(4.10.0) /Users/xperience/GHA-Actions-OpenCV/_work/opencv-python/opencv-python/opencv/modules/imgproc/src/color.cpp:196: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


: 